In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
%matplotlib inline

Load Data
====

In [2]:
al_performance_2015 = pd.read_csv("2015_AL.csv").set_index("District")
al_performance_2016 = pd.read_csv("2016_AL.csv").set_index("District")
al_performance_2017 = pd.read_csv("2017_AL.csv").set_index("District")
al_performance_2018 = pd.read_csv("2018_AL.csv").set_index("District")


ol_performance_2016 = pd.read_csv("ol_failed_2016.csv").set_index("District")
ol_performance_2017 = pd.read_csv("ol_failed_2017.csv").set_index("District")

school_by_func_grade_2016= pd.read_csv("schools_by_func_grade_2016.csv").set_index("District")
school_by_func_grade_2017= pd.read_csv("schools_by_func_grade_2017.csv").set_index("District")

school_by_func_grade_2016['1AB'] = school_by_func_grade_2016['1AB']/school_by_func_grade_2016['Total']
school_by_func_grade_2016['1C'] = school_by_func_grade_2016['1C']/school_by_func_grade_2016['Total']
school_by_func_grade_2016['Type 2'] = school_by_func_grade_2016['Type 2']/school_by_func_grade_2016['Total']
school_by_func_grade_2016['Type 3'] = school_by_func_grade_2016['Type 3']/school_by_func_grade_2016['Total']

school_by_func_grade_2017['1AB'] = school_by_func_grade_2017['1AB']/school_by_func_grade_2017['Total']
school_by_func_grade_2017['1C'] = school_by_func_grade_2017['1C']/school_by_func_grade_2017['Total']
school_by_func_grade_2017['Type 2'] = school_by_func_grade_2017['Type 2']/school_by_func_grade_2017['Total']
school_by_func_grade_2017['Type 3'] = school_by_func_grade_2017['Type 3']/school_by_func_grade_2017['Total']

al_perf_with_demographics_2016 = al_performance_2016\
    .merge(school_by_func_grade_2016, left_index=True, right_index=True)\
    .merge(ol_performance_2016, left_index=True, right_index=True)

al_perf_with_demographics_2017 = al_performance_2017\
    .merge(school_by_func_grade_2017, left_index=True, right_index=True)\
    .merge(ol_performance_2017, left_index=True, right_index=True)

           

Linear Regression
======
Train on 2016, test on 2017

Only with demographic and past year results

In [3]:

df_2016 = al_perf_with_demographics_2016
cols = ["prev_pass_all_perc","prev_fail_all_perc",  '1AB', '1C', 'Type 2', 'Type 3', "ol_qualified_for_al_perc", "ol_failed_all_perc"]
X = df_2016[cols]
Y = df_2016["pass_all_perc"]
regr = linear_model.LinearRegression(normalize=True)

regr.fit(X, Y)
pred = regr.predict(X)
r_sq = regr.score(X, Y)
print(r_sq)


0.6971139573440708


In [4]:
df_2017 =al_perf_with_demographics_2017
x2 = df_2017[cols]
y2 = df_2017["pass_all_perc"]

print(regr.score(x2,y2))

-0.11903639781811748


This is due to lack of data, making the model overfit to the given small amount.


With District included as one-hot-encoding
=====

In [5]:
district_enc = OneHotEncoder()
district_enc.fit(al_perf_with_demographics_2016.index.values[:, None])
district_col_names = list(district_enc.categories_[0])

In [6]:
df_2016 = al_perf_with_demographics_2016.reset_index() #Remove district from index and add it as a normal column
d = district_enc.transform(df_2016['District'].values[:, None]).todense() # Convert district names to one-hot-matrix
df_district_onehot = pd.DataFrame(data=d, columns=district_col_names)

df_2016 = pd.concat([df_2016, df_district_onehot], axis=1) # Combine two dataframes


df_2017 = al_perf_with_demographics_2017.reset_index() #Remove district from index and add it as a normal column
d = district_enc.transform(df_2017['District'].values[:, None]).todense() # Convert district names to one-hot-matrix
df_district_onehot = pd.DataFrame(data=d, columns=district_col_names)

df_2017 = pd.concat([df_2017, df_district_onehot], axis=1) # Combine two dataframes


In [7]:
cols = district_col_names + ["prev_pass_all_perc","prev_fail_all_perc",  '1AB', '1C', 'Type 2', 'Type 3', "ol_qualified_for_al_perc", "ol_failed_all_perc"]
X = df_2016[cols]
Y = df_2016["pass_all_perc"]
regr = linear_model.LinearRegression(normalize=True)

regr.fit(X, Y)
pred = regr.predict(X)
r_sq = regr.score(X, Y)
print(r_sq)

1.0


Just doing it for one year will overfit

In [8]:

x2 = df_2017[cols]
y2 = df_2017["pass_all_perc"]

print(regr.score(x2,y2))

-0.6978612284520458


And the predictions are bad

With multiple years
====

In [9]:
df_2016_2017 = pd.concat([al_perf_with_demographics_2016, al_perf_with_demographics_2017])
df_2016_2017 = df_2016_2017.reset_index()
d = district_enc.transform(df_2016_2017['District'].values[:, None]).todense() # Convert district names to one-hot-matrix
df_district_onehot = pd.DataFrame(data=d, columns=district_col_names)

df_2016_2017 = pd.concat([df_2016_2017, df_district_onehot], axis=1)


In [10]:
cols = district_col_names + ["prev_pass_all_perc","prev_fail_all_perc",  '1AB', '1C', 'Type 2', 'Type 3', "ol_qualified_for_al_perc", "ol_failed_all_perc"]
X = df_2016_2017[cols]
Y = df_2016_2017["fail_all_perc"]
regr = linear_model.LinearRegression(normalize=True)
regr.fit(X, Y)
pred = regr.predict(X)
r_sq = regr.score(X,Y)
print(r_sq)



0.9407664405283351


When trained with multiple years the accuracy improves for the given years

In [11]:

x2 = df_2016[cols]
y2 = df_2016["fail_all_perc"]

x3 = df_2017[cols]
y3 = df_2017["fail_all_perc"]
print(regr.score(x2,y2))
print(regr.score(x3,y3))

0.9369169053704236
0.9435632655386507


This seems to fit well, when tested with a data that was already included in the train set. 


# The following model is trained only using past results and no demographical data is used to train the model


In [12]:
df_new = pd.read_csv("2015_to_2018_AL.csv")

df_new
x1 = df_new[["num_sat","fail_all","fail_all_perc"]]
y1 = df_new[["pass_all_perc"]]

regr2 = linear_model.LinearRegression(normalize=True)
regr2.fit(x1,y1)             #trains model with data of AL number of students sat, failed students and failed perc from 2015-2018
print(regr2.score(x1,y1))
df_2016 = al_perf_with_demographics_2016
x3 = df_2016[["num_sat","fail_all","fail_all_perc"]]
y3 = df_2016[["pass_all_perc"]]
print(regr2.score(x3,y3))

0.5442729152515344
0.6045777904086118


In [13]:
df_new = pd.read_csv("2015_to_2018_AL_func.csv")

df_new
x1 = df_new[["num_sat","fail_all","fail_all_perc","1AB","1C","Type 2","Type 3"]]
y1 = df_new[["pass_all_perc"]]

regr3 = linear_model.LinearRegression(normalize=True)
regr3.fit(x1,y1)
print(regr3.score(x1,y1))
df_2016 = al_perf_with_demographics_2016
x3 = df_2016[["num_sat","fail_all","fail_all_perc","1AB","1C","Type 2","Type 3"]]
y3 = df_2016[["pass_all_perc"]]
print(regr3.score(x3,y3))

0.5983124317773196
0.22936080312115115


In [14]:
x1_new = sm.add_constant(x1)
est = sm.OLS(y1, x1_new)
est2 = est.fit()
print(est2.summary())

C:\Users\Randika\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


                            OLS Regression Results                            
Dep. Variable:          pass_all_perc   R-squared:                       0.598
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     14.26
Date:                Sat, 25 Apr 2020   Prob (F-statistic):           3.47e-11
Time:                        15:19:20   Log-Likelihood:                -149.23
No. Observations:                  75   AIC:                             314.5
Df Residuals:                      67   BIC:                             333.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const            77.8762      2.701     28.833